# Simulate in PyBullet

In [ ]:
from typing import Dict

import gtdynamics as gtd

import time
import matplotlib.pyplot as plt
import pybullet as p
import pybullet_data
import pandas as pd
import numpy as np

In [ ]:
import pybullet as p
p.connect(p.GUI);

In [ ]:
import pybullet_data
p.setAdditionalSearchPath(pybullet_data.getDataPath())

In [ ]:
# First, let's make sure we start with a fresh new simulation.
p.resetSimulation()
p.setGravity(0, 0, -9.8)
p.setRealTimeSimulation(0)
planeId = p.loadURDF(gtd.URDF_PATH+"/walls.urdf")

In [ ]:
file = gtd.SDF_PATH + "/spider_alt.sdf"
!ls {file}
robot = p.loadSDF(file)
robot_id = robot[0]  # loadSDF returns a list of objects; the first is the integer ID.

In [ ]:
joint_to_jid_map = {}
for i in range(p.getNumJoints(robot_id)):
    jinfo = p.getJointInfo(robot_id, i)
    joint_to_jid_map[jinfo[1].decode("utf-8")] = jinfo[0]

In [ ]:
def set_joint_angles(joint_angles: Dict[str, float], joint_vels: Dict[str, float]):
    """Actuate to the supplied joint angles using PD control."""
    for jid in joint_to_jid_map.values():
        p.setJointMotorControl2(robot_id, jid, p.VELOCITY_CONTROL, force=5000)

    for k, v in joint_angles.items():
        p.setJointMotorControl2(bodyUniqueId=robot_id,
                                jointIndex=joint_to_jid_map[k],
                                controlMode=p.POSITION_CONTROL,
                                targetPosition=v,
                                targetVelocity=joint_vels[k + '.1'])

In [ ]:
df = pd.read_csv('spider.csv')
print(df.columns)

In [ ]:
p.resetBasePositionAndOrientation(robot_id, [0, 0, 0.21], [0, 0, 0, 1])

In [ ]:
# parameters:
pos, orn = p.getBasePositionAndOrientation(robot_id)

print("Init Base\n\tPos: {}\n\tOrn: {}".format(pos,
                                               p.getEulerFromQuaternion(orn)))

max_traj_replays = 1
num_traj_replays = 0

t = 0
t_f = None
ts = []
all_pos_sim = []

link_dict = {}
link_to_num = {3: 0, 7: 1, 11: 2, 15: 3, 19: 4, 23: 5, 27: 6, 31: 7}
constrained = []
dt = 1.0/240

In [ ]:
import time
i = 0
while True:
    if num_traj_replays == max_traj_replays:
        break

    if (i == len(df) - 1):
        i = 0
        if num_traj_replays == 0:
            t_f = t
        num_traj_replays += 1

    jangles = df.loc[i][np.arange(32)]
    jvels = df.loc[i][np.arange(32, 64)]
    jaccels = df.loc[i][np.arange(64, 96)]
    jtorques = df.loc[i][np.arange(96, 128)]

    set_joint_angles(jangles, jvels)

    # Update body CoM coordinate frame.
    new_pos, new_orn = p.getBasePositionAndOrientation(robot_id)
    new_pos = np.array(new_pos)
    new_R = np.array(p.getMatrixFromQuaternion(new_orn)).reshape(3, 3)

    print(i)

    # Detect collision points and constrain them.
    cp = np.asarray(p.getContactPoints(bodyA=planeId, bodyB=robot_id))
    if cp.shape[0] > 1 and i > 1:
        new_cps = set(cp[:, 4])

        change = list(df.loc[i][np.arange(16, 24)] -
                      df.loc[i-1][np.arange(16, 24)])
        # Initial collision
        just_collided = [
            x for x in new_cps if x not in constrained and x in link_to_num.keys()]
        for x in just_collided:
            if (link_to_num[x] < 4 and change[link_to_num[x]] >= 0) or (link_to_num[x] >= 4 and change[link_to_num[x]] <= 0):
                link_dict[x] = p.createConstraint(robot_id, x, planeId, -1, p.JOINT_POINT2POINT, [
                                                  0, 0, 0], [0, 0, 0], p.getLinkState(robot_id, x)[0])
                constrained.append(x)

        # Wants to lift
        for x in constrained:
            if (link_to_num[x] < 4 and change[link_to_num[x]] <= 0) or (link_to_num[x] >= 4 and change[link_to_num[x]] >= 0) and link_dict.get(x) != None:
                numConstraints_before = p.getNumConstraints()
                p.removeConstraint(link_dict[x])
                if numConstraints_before != p.getNumConstraints():
                    constrained.remove(x)

    p.stepSimulation()
    # time.sleep(dt)

    ts.append(t)
    t += dt
    all_pos_sim.append(new_pos)
    i += 1


pos, orn = p.getBasePositionAndOrientation(robot_id)
print("Final Base\n\tPos: {}\n\tOrn: {}".format(pos,
                                                p.getEulerFromQuaternion(orn)))


In [ ]:
pos, orn = p.getBasePositionAndOrientation(robot_id)
print("Final Base\n\tPos: {}\n\tOrn: {}".format(pos,
                                                p.getEulerFromQuaternion(orn)))

fig, axs = plt.subplots(3, 1, sharex=True)
fig.subplots_adjust(hspace=0)

axs[0].plot(ts, [p[0] for p in all_pos_sim])
axs[0].axvline(x=t_f, color='k', linestyle='--')
axs[0].set_ylabel('x (m.)')

axs[1].plot(ts, [p[1] for p in all_pos_sim])
axs[1].axvline(x=t_f, color='k', linestyle='--')
axs[1].set_ylabel('y (m.)')

axs[2].plot(ts, [p[2] for p in all_pos_sim])
axs[2].axvline(x=t_f, color='k', linestyle='--')
axs[2].set_ylabel('z (m.)')

plt.xlabel("time (s.)")

plt.show()

In [ ]:
while True:
    p.stepSimulation()
    time.sleep(1. / 240)
p.disconnect()